# Build a strategy from scratch

Welcome to the third part of the Flower federated learning tutorial. In previous parts of this tutorial, we introduced federated learning with PyTorch and Flower ([part 1](https://flower.dev/docs/framework/tutorial-get-started-with-flower-pytorch.html)) and we learned how strategies can be used to customize the execution on both the server and the clients ([part 2](https://flower.dev/docs/framework/tutorial-use-a-federated-learning-strategy-pytorch.html)).

In this notebook, we'll continue to customize the federated learning system we built previously by creating a custom version of FedAvg (again, using [Flower](https://flower.dev/) and [PyTorch](https://pytorch.org/)).

> [Star Flower on GitHub](https://github.com/adap/flower) ⭐️ and join the Flower community on Slack to connect, ask questions, and get help: [Join Slack](https://flower.dev/join-slack) 🌼 We'd love to hear from you in the `#introductions` channel! And if anything is unclear, head over to the `#questions` channel.

Let's build a new `Strategy` from scratch!

## Preparation

Before we begin with the actual code, let's make sure that we have everything we need.

### Installing dependencies

First, we install the necessary packages:

In [3]:
!pip install -q flwr[simulation] torch torchvision

Now that we have all dependencies installed, we can import everything we need for this tutorial:

In [2]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

import flwr as fl

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cpu using PyTorch 2.0.1+cu117 and Flower 1.5.0


It is possible to switch to a runtime that has GPU acceleration enabled (on Google Colab: `Runtime > Change runtime type > Hardware acclerator: GPU > Save`). Note, however, that Google Colab is not always able to offer GPU acceleration. If you see an error related to GPU availability in one of the following sections, consider switching back to CPU-based execution by setting `DEVICE = torch.device("cpu")`. If the runtime has GPU acceleration enabled, you should see the output `Training on cuda`, otherwise it'll say `Training on cpu`.

### Data loading

Let's now load the CIFAR-10 training and test set, partition them into ten smaller datasets (each split into training and validation set), and wrap everything in their own `DataLoader`. We introduce a new parameter `num_clients` which allows us to call `load_datasets` with different numbers of clients.

In [4]:
NUM_CLIENTS = 10


def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


### Model training/evaluation

Let's continue with the usual model definition (including `set_parameters` and `get_parameters`), training and test functions:

In [5]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

### Flower client

To implement the Flower client, we (again) create a subclass of `flwr.client.NumPyClient` and implement the three methods `get_parameters`, `fit`, and `evaluate`. Here, we also pass the `cid` to the client and use it log additional details:

In [6]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

Let's test what we have so far before we continue:

In [7]:
# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=2,
    config=fl.server.ServerConfig(num_rounds=3),
    client_resources=client_resources,
)

INFO flwr 2023-08-25 17:37:10,127 | app.py:175 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-08-25 17:37:12,276	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-08-25 17:37:13,422 | app.py:210 | Flower VCE: Ray initialized with resources: {'GPU': 8.0, 'node:172.28.177.173': 1.0, 'object_store_memory': 41252398694.0, 'memory': 86255596954.0, 'CPU': 80.0}
INFO flwr 2023-08-25 17:37:13,424 | app.py:218 | No `client_resources` specified. Using minimal resources for clients.
INFO flwr 2023-08-25 17:37:13,425 | app.py:224 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2023-08-25 17:37:13,477 | app.py:270 | Flower VCE: Creating VirtualClientEngineActorPool with 80 actors
INFO flwr 2023-08-25 17:37:13,479 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-25 17:37:13,479 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-25 17:37:16,72

(DefaultActor pid=1995895) [Client 1] get_parameters
(DefaultActor pid=1995895) [Client 1] fit, config: {}
(DefaultActor pid=1995895) Epoch 1: train loss 0.064800925552845, accuracy 0.22866666666666666


DEBUG flwr 2023-08-25 17:37:22,769 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-25 17:37:22,778 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-08-25 17:37:22,779 | server.py:173 | evaluate_round 1: strategy sampled 2 clients (out of 2)


(DefaultActor pid=1995894) [Client 0] evaluate, config: {}
(DefaultActor pid=1995894) [Client 0] evaluate, config: {}


DEBUG flwr 2023-08-25 17:37:25,005 | server.py:187 | evaluate_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-25 17:37:25,007 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-08-25 17:37:25,008 | server.py:222 | fit_round 2: strategy sampled 2 clients (out of 2)


...

(DefaultActor pid=1995895) [Client 1] fit, config: {} [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(DefaultActor pid=1995894) [Client 1] fit, config: {}
(DefaultActor pid=1995895) Epoch 1: train loss 0.052503738552331924, accuracy 0.3873333333333333


DEBUG flwr 2023-08-25 17:37:35,730 | server.py:236 | fit_round 3 received 2 results and 0 failures
DEBUG flwr 2023-08-25 17:37:35,737 | server.py:173 | evaluate_round 3: strategy sampled 2 clients (out of 2)


(DefaultActor pid=1995895) Epoch 1: train loss 0.053061239421367645, accuracy 0.3828888888888889
(DefaultActor pid=1995894) [Client 1] evaluate, config: {}


DEBUG flwr 2023-08-25 17:37:38,009 | server.py:187 | evaluate_round 3 received 2 results and 0 failures
INFO flwr 2023-08-25 17:37:38,010 | server.py:153 | FL finished in 21.28000294417143
INFO flwr 2023-08-25 17:37:38,013 | app.py:225 | app_fit: losses_distributed [(1, 0.06246930694580078), (2, 0.05624225044250489), (3, 0.053368818402290345)]
INFO flwr 2023-08-25 17:37:38,014 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-08-25 17:37:38,015 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-08-25 17:37:38,016 | app.py:228 | app_fit: losses_centralized []
INFO flwr 2023-08-25 17:37:38,017 | app.py:229 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.06246930694580078
	round 2: 0.05624225044250489
	round 3: 0.053368818402290345

## Build a Strategy from scratch

Let’s overwrite the `configure_fit` method such that it passes a higher learning rate (potentially also other hyperparameters) to the optimizer of a fraction of the clients. We will keep the sampling of the clients as it is in `FedAvg` and then change the configuration dictionary (one of the `FitIns` attributes).

In [8]:
from typing import Callable, Union

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg


class FedCustom(fl.server.strategy.Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients

    def __repr__(self) -> str:
        return "FedCustom"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = Net()
        ndarrays = get_parameters(net)
        return fl.common.ndarrays_to_parameters(ndarrays)

    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""

        # Sample clients
        sample_size, min_num_clients = self.num_fit_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Create custom configs
        n_clients = len(clients)
        half_clients = n_clients // 2
        standard_config = {"lr": 0.001}
        higher_lr_config = {"lr": 0.003}
        fit_configurations = []
        for idx, client in enumerate(clients):
            if idx < half_clients:
                fit_configurations.append((client, FitIns(parameters, standard_config)))
            else:
                fit_configurations.append(
                    (client, FitIns(parameters, higher_lr_config))
                )
        return fit_configurations

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""

        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]
        parameters_aggregated = ndarrays_to_parameters(aggregate(weights_results))
        metrics_aggregated = {}
        return parameters_aggregated, metrics_aggregated

    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        if self.fraction_evaluate == 0.0:
            return []
        config = {}
        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        return [(client, evaluate_ins) for client in clients]

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        if not results:
            return None, {}

        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )
        metrics_aggregated = {}
        return loss_aggregated, metrics_aggregated

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate global model parameters using an evaluation function."""

        # Let's assume we won't perform the global model evaluation on the server side.
        return None

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients

The only thing left is to use the newly created custom Strategy `FedCustom` when starting the experiment:

In [9]:
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=2,
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=FedCustom(),  # <-- pass the new strategy here
    client_resources=client_resources,
)

INFO flwr 2023-08-25 18:08:31,298 | app.py:175 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)


(DefaultActor pid=1995894) [Client 0] fit, config: {}
(DefaultActor pid=1995894) Epoch 1: train loss 0.053061239421367645, accuracy 0.3828888888888889
(DefaultActor pid=1995895) [Client 0] evaluate, config: {}


2023-08-25 18:08:37,015	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-08-25 18:08:38,007 | app.py:210 | Flower VCE: Ray initialized with resources: {'CPU': 80.0, 'GPU': 8.0, 'object_store_memory': 40751050752.0, 'memory': 85085785088.0, 'node:172.28.177.173': 1.0}
INFO flwr 2023-08-25 18:08:38,010 | app.py:218 | No `client_resources` specified. Using minimal resources for clients.
INFO flwr 2023-08-25 18:08:38,011 | app.py:224 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2023-08-25 18:08:38,056 | app.py:270 | Flower VCE: Creating VirtualClientEngineActorPool with 80 actors
INFO flwr 2023-08-25 18:08:38,057 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-25 18:08:38,082 | server.py:272 | Using initial parameters provided by strategy
INFO flwr 2023-08-25 18:08:38,083 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-25 18:08:38,084 | server.py:104 | FL starting
DEBUG flwr 2023-08-25 18:0

(DefaultActor pid=2336359) [Client 0] fit, config: {'lr': 0.001}
(DefaultActor pid=2336359) Epoch 1: train loss 0.06447555869817734, accuracy 0.23444444444444446


DEBUG flwr 2023-08-25 18:08:44,961 | server.py:236 | fit_round 1 received 2 results and 0 failures
DEBUG flwr 2023-08-25 18:08:44,970 | server.py:173 | evaluate_round 1: strategy sampled 2 clients (out of 2)


...

(DefaultActor pid=2336359) [Client 1] fit, config: {'lr': 0.001} [repeated 2x across cluster]
(DefaultActor pid=2336358) [Client 1] fit, config: {'lr': 0.001}
(DefaultActor pid=2336359) Epoch 1: train loss 0.052343450486660004, accuracy 0.3888888888888889


DEBUG flwr 2023-08-25 18:08:58,629 | server.py:236 | fit_round 3 received 2 results and 0 failures
DEBUG flwr 2023-08-25 18:08:58,638 | server.py:173 | evaluate_round 3: strategy sampled 2 clients (out of 2)


(DefaultActor pid=2336359) Epoch 1: train loss 0.052768904715776443, accuracy 0.3837777777777778
(DefaultActor pid=2336358) [Client 0] evaluate, config: {}


DEBUG flwr 2023-08-25 18:09:00,985 | server.py:187 | evaluate_round 3 received 2 results and 0 failures
INFO flwr 2023-08-25 18:09:00,986 | server.py:153 | FL finished in 22.90086031332612
INFO flwr 2023-08-25 18:09:00,986 | app.py:225 | app_fit: losses_distributed [(1, 0.06039282703399658), (2, 0.05561759889125824), (3, 0.053774427771568295)]
INFO flwr 2023-08-25 18:09:00,987 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-08-25 18:09:00,988 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-08-25 18:09:00,989 | app.py:228 | app_fit: losses_centralized []
INFO flwr 2023-08-25 18:09:00,990 | app.py:229 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.06039282703399658
	round 2: 0.05561759889125824
	round 3: 0.053774427771568295

## Recap

In this notebook, we’ve seen how to implement a custom strategy. A custom strategy enables granular control over client node configuration, result aggregation, and more. To define a custom strategy, you only have to overwrite the abstract methods of the (abstract) base class `Strategy`. To make custom strategies even more powerful, you can pass custom functions to the constructor of your new class (`__init__`) and then call these functions whenever needed. 

## Next steps

Before you continue, make sure to join the Flower community on Slack: [Join Slack](https://flower.dev/join-slack/)

There's a dedicated `#questions` channel if you need help, but we'd also love to hear who you are in `#introductions`!

The [Flower Federated Learning Tutorial - Part 4](https://flower.dev/docs/framework/tutorial-customize-the-client-pytorch.html) introduces `Client`, the flexible API underlying `NumPyClient`.